<a href="https://colab.research.google.com/github/sbagency/colab-hacks/blob/main/Llam3_2_Groq_Inference_Chainlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chainlit

In [2]:
!pip install openai

In [3]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [4]:
import portpicker
port = portpicker.pick_unused_port()

In [5]:
import os
#print(port)
os.environ['PICKED_PORT'] = f"{port}"
!echo $PICKED_PORT

39651


In [6]:
from google.colab import output
output.serve_kernel_port_as_window(port)
output.serve_kernel_port_as_iframe(port)

# get_ipython().system_raw("python3 -m http.server 8888 &")

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
#response = await client.chat.completions.create(
#            model="gpt-4",
#            messages=messages,
#            tools=functions,
#            temperature=0.0,
#            tool_choice=None
#        )

#!python

Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> l=[1,2,3,4,5]
>>> del l[1:3]
>>> l
[1, 4, 5]
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> ^C


In [7]:
%%writefile app.py
import openai
import os
from google.colab import userdata

client = openai.AsyncOpenAI(
  #base_url = "https://integrate.api.nvidia.com/v1",
  #api_key = userdata.get('NVIDIA_API_KEY')
  base_url="https://api.groq.com/openai/v1",
  api_key=os.environ['GROQ_API_KEY'] #userdata.get('GROQ_API_KEY')
)

#model = "meta/llama-3.1-405b-instruct"
#model = "llama-3.1-70b-versatile"
model = "llama-3.2-90b-text-preview"
temperature=1
top_p=1
max_tokens=4096

async def llmm(messages):
  #messages=[{"role": "user","content": prompt}]
  completion = await client.chat.completions.create(
  model=model,
  messages=messages,
  temperature=temperature,
  top_p=top_p,
  max_tokens=max_tokens,
  stream=False
  )
  return completion.choices[0].message.content

async def llm(prompt):
  messages=[{"role": "user","content": prompt}]
  completion = await client.chat.completions.create(
  model=model,
  messages=messages,
  temperature=temperature,
  top_p=top_p,
  max_tokens=max_tokens,
  stream=False
  )
  return completion.choices[0].message.content

import chainlit as cl

system_message={"role": "system","content": "You are a helpful assistant!"}

@cl.on_chat_start
async def on_chat_start():
    cl.user_session.set("messages", [system_message])

MSG_MAX=3*2+1

@cl.on_message
async def main(message: cl.Message):
    # Your custom logic goes here...
    messages = cl.user_session.get("messages")
    messages.append({"role": "user","content": message.content})
    resp=await llmm(messages)
    messages.append({"role": "assistant","content": resp})
    if len(messages)>MSG_MAX:
      messages=[system_message]+messages[3:]
    print(len(messages),messages)
    cl.user_session.set("messages", messages)
    # Send a response back to the user
    await cl.Message(
        content=resp
    ).send()

Overwriting app.py


In [8]:
from google.colab.output import eval_js
external_url=eval_js(f"google.colab.kernel.proxyPort({port})")
print(external_url)

https://dt0fe3mg09w-496ff2e9c6d22116-39651-colab.googleusercontent.com/


In [ ]:
!echo $PICKED_PORT
!chainlit run --port $PICKED_PORT app.py

41117
2024-09-30 09:28:43 - Your app is available at http://localhost:41117
2024-09-30 09:28:50 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-09-30 09:28:56 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
3 [{'role': 'system', 'content': 'You are a helpful assistant!'}, {'role': 'user', 'content': 'message 1'}, {'role': 'assistant', 'content': "I'd be happy to help. What's on your mind? Do you have a question, or is there something specific you'd like to know or discuss?"}]
2024-09-30 09:29:03 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
5 [{'role': 'system', 'content': 'You are a helpful assistant!'}, {'role': 'user', 'content': 'message 1'}, {'role': 'assistant', 'content': "I'd be happy to help. What's on your mind? Do you have a question, or is there something specific you'd like to know or discuss?"}, {'role': 'user', 'content': 'message 2'}, {'role': 'assistant', 'co

In [9]:
!echo $PICKED_PORT
!nohup chainlit run --port $PICKED_PORT app.py &

39651
nohup: appending output to 'nohup.out'


In [15]:
!cat nohup.out

2024-09-30 10:44:55 - Your app is available at http://localhost:45793
2024-09-30 10:45:29 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-09-30 10:46:02 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-09-30 10:46:13 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
3 [{'role': 'system', 'content': 'You are a helpful assistant!'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Hello. How can I assist you today?'}]
2024-09-30 10:46:43 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-09-30 10:47:10 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-30 10:50:07 - Your app is available at http://localhost:39651
2024-09-30 10:50:17 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-09-30 10:50:23 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "

In [11]:
from IPython.display import IFrame

IFrame(src=external_url,width="100%",height=640)